<a href="https://colab.research.google.com/github/SabinaDDD/Taxi/blob/main/aparu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Агрегатор такси применяет для своих водителей рейтинговую систему, в которой учитываются среднее количество заказов в день (за последние 14 дней) , наличие ИП, и другие факторы. Активные водители нарабатывающие в день от 35 заказов могут заработать бонусный выходной день на время которого, среднее количество заказов замораживается. Необходимо для активных , постоянно работающих в данном такси водителей разработать схему для  при которой будет учитываться их многолетний стаж и отработтанные заказы в работе такси для более быстрого  получения бонусных дней.  
Я предлагаю в таблицу в личными данными о водителях , прописать для каждого персональное количество заказов достаточных для получения бонусного дня.

Импорт библиотек для работы

In [ ]:
import pandas as pd
import seaborn as sns
from datetime import datetime as dt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Считываем таблицу с данными о водителях

In [ ]:
drivers_df = pd.read_excel('/content/drive/MyDrive/data/drivers.xlsx')
drivers_df.head()

,driver_id,driver_name,reg_date
0,1001,Аманжолов Берик,2016-01-01
1,1002,Кремер Александр,2020-02-02
2,1003,Жоламанов Ерасыл,2018-03-03
3,1004,Кривко Петр,2021-01-01


Делаем преобразование типов

In [ ]:
drivers_df['reg_date'] = pd.to_datetime(drivers_df['reg_date'])#
drivers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   driver_id    4 non-null      int64         
 1   driver_name  4 non-null      object        
 2   reg_date     4 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 224.0+ bytes


Заводим переменную с текущей датой

In [ ]:
current_dt = dt.now() # текущая дата
type(current_dt)

datetime.datetime

Добавляем столбец со стажем в Апару ,
 отняв от текущей даты дату регистрации

In [ ]:
drivers_df['exp_delta'] = drivers_df['reg_date'].apply(lambda x: current_dt - x) #применяем лямбда функцию
drivers_df.head()# получаем временной промежуток в днях

,driver_id,driver_name,reg_date,exp_delta
0,1001,Аманжолов Берик,2016-01-01,2705 days 10:38:32.239918
1,1002,Кремер Александр,2020-02-02,1212 days 10:38:32.239918
2,1003,Жоламанов Ерасыл,2018-03-03,1913 days 10:38:32.239918
3,1004,Кривко Петр,2021-01-01,878 days 10:38:32.239918


Переведем временной промежуток в годы, для этого применим лямбда функцию

In [ ]:
drivers_df['exp_delta'] = drivers_df['exp_delta'].apply(lambda x: round(x.total_seconds()/(3600*24*365), 2))
drivers_df.head()

,driver_id,driver_name,reg_date,exp_delta
0,1001,Аманжолов Берик,2016-01-01,7.41
1,1002,Кремер Александр,2020-02-02,3.32
2,1003,Жоламанов Ерасыл,2018-03-03,5.24
3,1004,Кривко Петр,2021-01-01,2.41


Считываем другую таблицу с количеством выполненных заказов

In [ ]:
orders_df = pd.read_excel('/content/drive/MyDrive/data/orders.xlsx')
orders_df.head()

,driver_id,date,orders_cnt
0,1001,2023-01-01,30
1,1002,2023-01-01,25
2,1003,2023-01-01,10
3,1004,2023-01-01,2
4,1001,2023-01-02,36


Также приводим типы

In [ ]:
orders_df.date = pd.to_datetime(orders_df.date)
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   driver_id   160 non-null    int64         
 1   date        160 non-null    datetime64[ns]
 2   orders_cnt  160 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 3.9 KB


In [ ]:
orders_df.describe()

,driver_id,orders_cnt
count,160.000000,160.000000
mean,1002.500000,22.031250
std,1.121544,14.274674
min,1001.000000,0.000000
25%,1001.750000,10.000000
50%,1002.500000,20.000000
75%,1003.250000,35.000000
max,1004.000000,61.000000


Создаем Series со средним количеством заказов в день

In [ ]:
avg_cnt = orders_df[['driver_id', 'orders_cnt']].groupby(by='driver_id').agg('mean')['orders_cnt']
avg_cnt

driver_id
1001    31.400
1002    24.325
1003    16.750
1004    15.650
Name: orders_cnt, dtype: float64

In [ ]:
avg_cnt.loc[1001] #проверим доступ к числу с количеством для дальнейшего использования

31.4

Создаем Series со стажем водителей , в индексах которого id водителя

In [ ]:
exp_delta = drivers_df[['driver_id','exp_delta']].groupby(by='driver_id').agg('mean')['exp_delta']
exp_delta

driver_id
1001    7.41
1002    3.32
1003    5.24
1004    2.41
Name: exp_delta, dtype: float64

In [ ]:
exp_delta.loc[1001] # Проверим доступ к данным через id водителя для дальнейшего использования

7.41

Напишем функцию для вычисления коэффициента, который в дальнейшем будет использоваться для подсчета количества заказов достаточных для выделения водителю выходного дня

In [ ]:
def orders_for_bonus(dr_id):
    koef = 1
    exper = exp_delta.loc[dr_id] # вытаскиваем из данных стаж водителя в Апару
    if avg_cnt.loc[dr_id] >= 20: # берем среднее количество заказов в день, и если оно больше 20
        # (взяла грубо 8ми часовой рабочий день , если на 1 заказ примерно уходит 30 мин, то в день 16 заказов -> округлила до 20 )
        if 0 <= exper <3: # Если при этом стаж от 0 до 3 лет то коэф оставляем 1
            koef = 1
        elif 3 <= exper <6: # Если стаж от 3 до 6 лет, то коэф берем 0.9
            koef = 0.9
        elif 6 <= exper < 9: # Если стаж от 6 до 9 лет, то коэф берем 0.8
            koef = 0.8
        elif 9 <= exper < 12:# Если стаж от 9 до 12 лет, то коэф берем 0.7
            koef = 0.7
    return koef

In [ ]:
orders_for_bonus(1001) # проверим как работает функция , введем в аргументы id первого водителя

0.8

Применим функцию orders_for_bonus к столбцу с id водителей

In [ ]:
drivers_df['koef'] = drivers_df['driver_id'].apply(orders_for_bonus)
drivers_df.head()

,driver_id,driver_name,reg_date,exp_delta,koef
0,1001,Аманжолов Берик,2016-01-01,7.41,0.8
1,1002,Кремер Александр,2020-02-02,3.32,0.9
2,1003,Жоламанов Ерасыл,2018-03-03,5.24,1.0
3,1004,Кривко Петр,2021-01-01,2.41,1.0


Как видим функция сработала и у первого водителя, коэффициент для баллов ниже а значит ему нужно будет меньше заказов для получения бонусного выходного дня

Я намеренно в таблицу ексель с заказами вбивала 1му водителю больше заказов , сделала больше стаж , 2му водителю чуть меньше заказов и стажа ,
и значительно меньше для 3го и 4го как заказов так и стажа

Теперь создадим столбец с персональным количеством заказов для получения бонусного выходного дня, умножив стандартные 35 заказов на коэффицент

In [ ]:
drivers_df['orders_for_bonus'] = round(drivers_df['koef'] * 35)
drivers_df.head()

,driver_id,driver_name,reg_date,exp_delta,koef,orders_for_bonus
0,1001,Аманжолов Берик,2016-01-01,7.41,0.8,28.0
1,1002,Кремер Александр,2020-02-02,3.32,0.9,32.0
2,1003,Жоламанов Ерасыл,2018-03-03,5.24,1.0,35.0
3,1004,Кривко Петр,2021-01-01,2.41,1.0,35.0


В столбце orders_for_bonus видим что количество заказов для получения выходного бонусного дня отличается. Для первого водителя достаточно выполнить 28 заказов, для второго водителя это - 32 заказа , для остальных остается стандартное количество 35 заказов.